**Problem Statement**

It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?
Take this monstrosity as the DataFrame to use in the following puzzles:

In [0]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',

'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})
print('\nData before Cleaning\n','-'*20, sep='')
df


Data before Cleaning
--------------------


From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris       10045.0      [23, 47]               KLM(!)
1      MAdrid_miLAN           NaN            []    <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis           NaN          [13]       12. Air France
4   Brussels_londOn       10085.0      [67, 32]          "Swiss Air"

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   From_To       5 non-null      object 
 1   FlightNumber  3 non-null      float64
 2   RecentDelays  5 non-null      object 
 3   Airline       5 non-null      object 
dtypes: float64(1), object(3)
memory usage: 288.0+ bytes


**1.** Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [0]:
df=df.fillna(value=10,axis=1)
df

From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris       10045.0      [23, 47]               KLM(!)
1      MAdrid_miLAN          10.0            []    <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis          10.0          [13]       12. Air France
4   Brussels_londOn       10085.0      [67, 32]          "Swiss Air"

In [0]:
x=df['FlightNumber']
for i in range(0,len(x)):
  if(x[i]==10):
    x[i]=x[i-1]+10

In [0]:
df

From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris       10045.0      [23, 47]               KLM(!)
1      MAdrid_miLAN       10055.0            []    <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis       10075.0          [13]       12. Air France
4   Brussels_londOn       10085.0      [67, 32]          "Swiss Air"

In [0]:
df['FlightNumber']=df['FlightNumber'].astype(int)
df

From_To  FlightNumber  RecentDelays              Airline
0      LoNDon_paris         10045      [23, 47]               KLM(!)
1      MAdrid_miLAN         10055            []    <Air France> (12)
2  londON_StockhOlm         10065  [24, 43, 87]  (British Airways. )
3    Budapest_PaRis         10075          [13]       12. Air France
4   Brussels_londOn         10085      [67, 32]          "Swiss Air"

In [0]:
df['FlightNumber'].dtypes

dtype('int64')

**2.** The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [0]:
x=[s.split('_') for s in df['From_To']]
tdf=pd.DataFrame(x,columns=['From','To'])
#tdf is the temporary dataframe
tdf

From         To
0    LoNDon      paris
1    MAdrid      miLAN
2    londON  StockhOlm
3  Budapest      PaRis
4  Brussels     londOn

**3.** Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [0]:
tdf['From'] = [x.lower() for x in tdf['From']]
tdf['To'] = [x.lower() for x in tdf['To']]
tdf

From         To
0    london      paris
1    madrid      milan
2    london  stockholm
3  budapest      paris
4  brussels     london

In [0]:
tdf['From']=[x[0].upper()+x[1:] for x in tdf['From']]
tdf['To']=[x[0].upper()+x[1:] for x in tdf['To']]
tdf

From         To
0    London      Paris
1    Madrid      Milan
2    London  Stockholm
3  Budapest      Paris
4  Brussels     London

**4.** Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [0]:
del df['From_To']
df=pd.concat([df, tdf], axis=1)
df

FlightNumber  RecentDelays              Airline      From         To
0         10045      [23, 47]               KLM(!)    London      Paris
1         10055            []    <Air France> (12)    Madrid      Milan
2         10065  [24, 43, 87]  (British Airways. )    London  Stockholm
3         10075          [13]       12. Air France  Budapest      Paris
4         10085      [67, 32]          "Swiss Air"  Brussels     London

**5.** In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each

second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [0]:
delay_data=[i for i in df['RecentDelays']]
delays=pd.DataFrame(delay_data,columns=['delay_1','delay_2','delay_3'])
delays

delay_1  delay_2  delay_3
0     23.0     47.0      NaN
1      NaN      NaN      NaN
2     24.0     43.0     87.0
3     13.0      NaN      NaN
4     67.0     32.0      NaN

In [0]:
df=pd.concat([df, delays], axis=1)
df

FlightNumber  RecentDelays              Airline  ... delay_1 delay_2  delay_3
0         10045      [23, 47]               KLM(!)  ...    23.0    47.0      NaN
1         10055            []    <Air France> (12)  ...     NaN     NaN      NaN
2         10065  [24, 43, 87]  (British Airways. )  ...    24.0    43.0     87.0
3         10075          [13]       12. Air France  ...    13.0     NaN      NaN
4         10085      [67, 32]          "Swiss Air"  ...    67.0    32.0      NaN

[5 rows x 8 columns]

In [0]:
del df['RecentDelays']
df

FlightNumber              Airline      From  ... delay_1  delay_2  delay_3
0         10045               KLM(!)    London  ...    23.0     47.0      NaN
1         10055    <Air France> (12)    Madrid  ...     NaN      NaN      NaN
2         10065  (British Airways. )    London  ...    24.0     43.0     87.0
3         10075       12. Air France  Budapest  ...    13.0      NaN      NaN
4         10085          "Swiss Air"  Brussels  ...    67.0     32.0      NaN

[5 rows x 7 columns]

In [0]:
df['Airline'] = df['Airline'].str.replace('[^a-zA-Z\s]','')
# To keep:
# Letters a-z, A-Z
# Dots (.)
# Numbers (\d)
# Non white space (\s)
# ^ means other than the provided patterns
print('\nData after Cleaning\n','-'*20, sep='')
df



Data after Cleaning
--------------------


FlightNumber           Airline      From  ... delay_1  delay_2  delay_3
0         10045               KLM    London  ...    23.0     47.0      NaN
1         10055       Air France     Madrid  ...     NaN      NaN      NaN
2         10065  British Airways     London  ...    24.0     43.0     87.0
3         10075        Air France  Budapest  ...    13.0      NaN      NaN
4         10085         Swiss Air  Brussels  ...    67.0     32.0      NaN

[5 rows x 7 columns]